In [5]:
# Ejercicio 1
from numpy.random import uniform
from numpy import bincount

def binom_probs(n, p):
    p0 = (1-p)**n
    ps = [p0]

    for j in range(1, n + 1):
        p0 *= ((n-j+1)/j) * (p / (1-p))
        ps.append(p0)

    return ps

def binom_prob_from_frecs(n, frecuencies):
    '''
    Sirve para estimar parámetro "p" de una distribución binomial a partir del n y sus frecuencias.
    '''
    m = sum(frecuencies)

    return sum(i*x for i, x in enumerate(frecuencies))/(n*m)

def optimized_bin(n, p):
    p0 = (1-p)**n
    F = p0
    
    max_bin = int(n * max(p, (1-p)))
    
    for j in range(1, max_bin + 1):
        p0 *= ((n-j+1)/j) * (p / (1-p))
        F += p0

    u = uniform()
    if (u >= F):
        j = max_bin + 1
        
        while(u >= F):
            p0 *= ((n-j+1)/j) * (p / (1-p))
            F += p0
            j += 1
            
        return j-1
    else:
        j = max_bin
        
        while(u < F):
            F -= p0
            p0 *= j/((n-j+1) * (p / (1-p)))
            j -= 1
            
        return j+1

# a)
print("a)")
Ns = [10, 44, 67, 52, 27]
m = sum(Ns)

n = 4
p_estimation = sum(i*x for i, x in enumerate(Ns))/(n*m)
print(f"Estimación del parámetro p: {p_estimation}")
ps = binom_probs(n, p_estimation)

# b)
print("b)")

# NOTE: m fue definida más arriba
def pearson_statistic(Ns, ps):
    return sum( (Ni - m*pi)**2/(m * pi) for Ni, pi in zip(Ns, ps))

t = pearson_statistic(Ns, ps)

print(f"Valor del estadístico bajo la muestra original: {t}")

# d)
print("d)")

# NOTE: m fue definida más arriba
# Versión naïve
def estimate_pvalue(t, n, p, num_of_sims=10):
    assertions = 0

    for _ in range(num_of_sims):
        sample = [optimized_bin(n, p) for _ in range(m)]
        Ms = bincount(sample)

        p_sim = binom_prob_from_frecs(n, Ms)
        ps_obs = binom_probs(n, p_sim)
        
        t_obs = pearson_statistic(Ms, ps_obs)
        
        assertions += int(t_obs >= t)
    
    return assertions/num_of_sims 

pvalue_estimation = estimate_pvalue(t, n, p_estimation, 10000)
print(f"Estimación del p-valor: {pvalue_estimation}")

from scipy.stats import multinomial
# Versión tryhard
# NOTE: Utiliza los valores de n y m de más arriba
def pvalue_estimation2(t, ps, num_of_sims):
    assertions = 0

    samples = multinomial(m, ps).rvs(num_of_sims)
    
    for Ms in samples:
        p_sim = binom_prob_from_frecs(n, Ms)
        ps_obs = binom_probs(n, p_sim)
        
        t_obs = pearson_statistic(Ms, ps_obs)
        assertions += int(t_obs >= t)

    return assertions/num_of_sims

p2 = pvalue_estimation2(t, ps, 10000)
print(f"Estimación del p-valor (usando multinomial): {p2}")


a)
Estimación del parámetro p: 0.5525
b)
Valor del estadístico bajo la muestra original: 6.441878945232575
d)
Estimación del p-valor: 0.095
Estimación del p-valor (usando multinomial): 0.0895
